# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
</div>
****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [25]:
n_b = len(data[data.race=='b'])
n_w = len(data[data.race!='w'])
call_b = sum(data[data.race=='b'].call)
call_w = sum(data[data.race=='w'].call)
prob_w = call_w/n_w
prob_b = call_b/n_b

n = n_b+n_w
p = (call_w+call_b)/n
q = 1-p

print(n*p*q)

360.446817248


1.  The CLT applies to this problem.  The normal distribution can be used to approximate the binomial distribution since n*p*(1-p) > 5.  We can test the proportions using the normal distribution

2. 
HO: prob_b==prob_w
HA: prob_b!=prob_w
critical value is 1.96 for 95% confidence

In [49]:
pooled_std = (p*(1-p))**0.5

t = (prob_b-prob_w)/(pooled_std*(1/n_b+1/n_w)**0.5)
t

-4.1084121524343464

The t-statistic is lower than the left tail critical value and we reject the null hypothesis that the mean callback rate for black sounding named applicants is the same as for white sounding named applicants

3.  Stats as follows

In [52]:
margin = 1.96*pooled_std/n**0.5
lower = p - margin
upper = p + margin

p_value = stats.norm.cdf(t)

print("p(callback) for the entire sample- "+str(p))
print("pooled std deviation for the entire sample- "+str(pooled_std))
print("p(callback) for race == b- "+str(prob_b))
print("p(callback) for race == w- "+str(prob_w))
print("margin of error for the (overall) mean- "+str(margin))
print("confidence interval for the (overall) mean- "+str(lower)+" to "+str(upper))
print("p value for "+str(t)+" is "+str(p_value))

p(callback) for the entire sample- 0.0804928131417
pooled std deviation for the entire sample- 0.272054627188
p(callback) for race == b- 0.064476386037
p(callback) for race == w- 0.0965092402464
margin of error for the (overall) mean- 0.00764095615545
confidence interval for the (overall) mean- 0.0728518569862 to 0.0881337692971
p value for -4.10841215243 is 1.99194341879e-05


4.  The mean callback rate for applicants whose name "sounds black" is, at high confidence, lower than that for applicants whose name "sounds white".  Americas long history of racial discrimination is a likely explanation for this.

5.  No, this simple analysis only looks at the sample means for callback rates for "black" and "white" and is less than dispositive.  A more thorough analysis would require controlling for other factors with a more elaborate model (likely linear regression) that examines many effects together.  I believe these have been run and have concluded that even controlling for other factors discrimination against "black named" applicants is highly likely.